In [5]:
import soepy
import json
from soepy.python.soepy_config import TEST_RESOURCES_DIR

fname = TEST_RESOURCES_DIR + "/regression_vault.soepy.json"
len(json.load(open(fname)))

100

In [25]:
import collections
import operator
Person = collections.namedtuple('Person', 'name age gender')
bob = Person(name='Bob', age=30, gender='male')
print('\nRepresentation:', bob)




Representation: Person(name='Bob', age=30, gender='male')


TypeError: 'Person' object is not callable